<a href="https://colab.research.google.com/github/parkhy0106/HolyMoly/blob/master/Autogluon-credit-score%2BEDA/FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Environment SETUP

##AutoGluonPreVersion

In [ ]:
# Uninstall mkl for faster neural-network training time
!pip uninstall -y mkl
# Upgrade pip to ensure the latest package versions are available
!pip install -U pip
# Upgrade setuptools to be compatible with namespace packages
!pip install -U setuptools
!pip install -U "mxnet<2.0.0"
# Install pre-release, frozen to a particual pre-release for stability
!pip install --pre "autogluon==0.0.16b20201214"
!pip install -U ipykernel

Uninstalling mkl-2019.0:
  Successfully uninstalled mkl-2019.0
     |████████████████████████████████| 1.6MB 7.5MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 785 kB 7.6 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 56.1.0
    Uninstalling setuptools-56.1.0:
      Successfully uninstalled setuptools-56.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


     |████████████████████████████████| 46.9 MB 100 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 309 kB 7.5 MB/s 
     |████████████████████████████████| 41 kB 585 kB/s 
     |████████████████████████████████| 242 kB 30.2 MB/s 
     |████████████████████████████████| 699 kB 40.2 MB/s 
     |████████████████████████████████| 206 kB 56.3 MB/s 
     |████████████████████████████████| 964 kB 55.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 101 kB 12.8 MB/s 
     |████████████████████████████████| 131 kB 62.6 MB/s 
     |████████████████████████████████| 119 kB 69.2 MB/s 
     |████████████████████████████████| 997 kB 58.1 MB/s 
     |████████████████████████████████| 2.1 MB 54.0 MB/s 
     |████████████

     |████████████████████████████████| 120 kB 10.0 MB/s 
  Attempting uninstall: ipykernel
    Found existing installation: ipykernel 4.10.1
    Uninstalling ipykernel-4.10.1:
      Successfully uninstalled ipykernel-4.10.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipykernel~=4.10, but you have ipykernel 5.5.5 which is incompatible.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##SDV(Need to be restarted and check numpy version which is 1.20)
######_*NOT COMPATIBLE WITH TENSORFLOW AND PYTORCH_*

##Auto preprossesing tool by Kaggler Sensei
######(https://github.com/jeongyoonlee)현재 비활성화됨

In [ ]:
!git clone https://github.com/jeongyoonlee/Kaggler

Cloning into 'Kaggler'...
remote: Enumerating objects: 1392, done.
remote: Counting objects: 100% (157/157), done.
remote: Compressing objects: 100% (122/122), done.
remote: Total 1392 (delta 72), reused 70 (delta 32), pack-reused 1235
Receiving objects: 100% (1392/1392), 2.07 MiB | 13.79 MiB/s, done.
Resolving deltas: 100% (865/865), done.


##My Own functions for This competition(굉장히 편협한 코드. 시간이 없어서 모듈화 못 함)

In [ ]:
directory = '/content/drive/MyDrive/mtfk/%~dp0/open/'
index = 'index'
import pandas as pd
train = pd.read_csv(directory+"train.csv",index_col=index)

In [ ]:
NaN=train['occyp_type'][0]

In [ ]:
def adj_col(a):
  a['occyp_type'] = a['occyp_type'].replace(NaN,'others')
  a = a.drop('FLAG_MOBIL',axis=1)
  a['begin_month'] = -a['begin_month']*30
  
  for x in a.index:
    if a['DAYS_EMPLOYED'][x] == 365243:
      a['occyp_type'][x] = 'beksu'
      a["DAYS_EMPLOYED"][x] = 0
  a["DAYS_EMPLOYED"] = -a["DAYS_EMPLOYED"]
  a["DAYS_BIRTH"] = -a["DAYS_BIRTH"]
 
 
  return a

In [ ]:
def split_col(a,b,num):
 trains=[]
 c=[]
 e=[]
 f=[]
 for x in a[b].unique():
  if len(x) >= num:
   print(x)
   trains.append(a[a[b]==x])
   e.append(x)
  else:
   c.append(a[a[b]==x])
   f.append(x)
 d = pd.concat(c)
 trains.append(d)
 e.append(f)
 return (trains, e)

In [ ]:
def models(a):
  d = []
  e = []
  f = []
  for x in range(len(a)):
    b = a[x].sample(int(len(a[x])/10))
    c = a[x].drop(b.index)
    fit_args = {'train_data': c, 'label': 'credit'}    
    d.append(task.fit(**fit_args))
    e.append(d[x].evaluate(b))
    d[x].persist_models()
    d[x].refit_full()
    d[x].distill()
    f.append(d[x].leaderboard(b, silent=True))
  return (d,f)

In [ ]:
def fold(a, n):
  b = []
  c = int(len(a)/n)
  for x in range(n-1):
    b.append(a.sample(c))
    a = a.drop(b[x].index)
  b.append(a)
  return b

In [ ]:
def fold_models(a):
  b = a
  d = []
  e = []
  f = []
  for x in range(len(b)):
    c = pd.concat(a).drop(b[x].index)
    fit_args = {'train_data': c, 'label': 'credit'}    
    d.append(task.fit(**fit_args,output_directory='/contect/Autogluon',eval_metric='log_loss'))
    e.append(d[x].evaluate(b[x]))
    d[x].persist_models()
    d[x].refit_full()
    #d[x].distill()
    f.append(d[x].leaderboard(b[x], silent=True))
  return (d,f)

In [ ]:
def check_duplicate(df, *args):
  check_list = list(set(df.columns.tolist())- set(args))
  
  # 중복되는 데이터 모두 저장하는 데이터 프레임  [boolean Seires]
  dup_pair_df = df.duplicated(subset=check_list, keep=False) # 모든 중복 샘플을 모두 True 처리 
  
  sum_dup = dup_pair_df.sum()# 중복 데이터 개수 확인
  print(f"{args}을(를) 제외하고 중복되는 데이터 개수는", sum_dup, "입니다")
  

  # 중복이 있을 경우 해당 데이터들의 index 반환
  if dup_pair_df.sum() != 0:
    print("중복 데이터 비율:", round((sum_dup/len(df)*100),2))
    print()
    return df.loc[dup_pair_df].index


In [ ]:
def group_same(df):
  a = 0
  b = []
  c = []
  for x in df['DAYS_BIRTH'].unique():
    for y in df['income_total'].unique():
      a = df[df['DAYS_BIRTH']==x]
      if a.sum !=0:
        pass
      b.append(a[a['income_total']==y])
      #출생일과 수입이 같으면 중복데이터라고 처리하기(너무 많은 값을 검사하면 시간이 오래 걸리고
      #그렇다고 query를 쓰자고 하니 코드 중복이 많아져서 디버깅하는 시간이 부족한 나에게는 모자람.)
      #라고해서 막 짰더니 실행속도가 너무 느려짐
  for x in range(len(b)):
    if len(b[x])!= 0:
      c.append(x)
  d = [] 
  for x in c:
    d.append(b[x])
  return d

#MODULE LOADING

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from autogluon.tabular import TabularPrediction as task

In [ ]:
cd Kaggler

In [ ]:
from kaggler.preprocessing import LabelEncoder
from kaggler.model import AutoLGB

In [ ]:
import kaggler
print(kaggler.__version__)

In [ ]:
cd ..

In [ ]:
from sklearn.metrics import log_loss

In [ ]:
directory = '/content/drive/MyDrive/mtfk/%~dp0/open/'  # directory where you have downloaded the data CSV files from the competition
label_column = 'credit'  # name of target variable to predict in this competition
eval_metric = log_loss  # Optional: specify that competition evaluation metric is  Log_loss
output_directory = '/content/drive/MyDrive/mtfk/results' 
index = 'index'
target_col = 'credit'

# DATA PROCESS SECTION

## Data Loading

In [ ]:
train = pd.read_csv(directory+"train.csv",index_col=index)

In [ ]:
test = pd.read_csv(directory+"test.csv",index_col=index)

In [ ]:
sub = pd.read_csv(directory+'sample_submission.csv', index_col=index) 

##Data Cleansing

In [ ]:
train = adj_col(train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
test = adj_col(test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


nan을 beksu(days_employed가 0인)와 others(고용상태이긴 한 사람들)로 나누어 표기하고 Days_emplyed가 양수(365243)면 0으로 치환 후 부호 양수로 변화
그 외에도 이런저런 처리를 추가할 수 있도록 함수화

### Overlapped data controling

일단 아예 완전히 일치하는 데이터들을 드랍하는 처리를 한 후에

In [ ]:
train.drop_duplicates(inplace=True)

##################################여기서부터 시작#####################################

온전히 일치하는데 credit값이 흔들리는 불안정한 객체들의 갯수를 확인

In [ ]:
train_c = train.copy()

In [ ]:
train = train_c.copy()

In [ ]:
check_duplicate(train_c, 'credit')

('credit',)을(를) 제외하고 중복되는 데이터 개수는 4497 입니다
중복 데이터 비율: 17.0



Int64Index([    1,     2,    19,    21,    24,    25,    29,    34,    48,
               50,
            ...
            26391, 26393, 26397, 26403, 26428, 26430, 26431, 26432, 26446,
            26451],
           dtype='int64', name='index', length=4497)

In [ ]:
credit_noise = train_c.loc[check_duplicate(train_c, 'credit')]
credit_noise.query('income_total==247500.0').query('begin_month==150.0')

('credit',)을(를) 제외하고 중복되는 데이터 개수는 4497 입니다
중복 데이터 비율: 17.0



,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,work_phone,phone,email,occyp_type,family_size,begin_month,credit
index,,,,,,,,,,,,,,,,,,
1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,11380,1540,0,0,1,Laborers,3.0,150.0,1.0
5167,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,11380,1540,0,0,1,Laborers,3.0,150.0,0.0


In [ ]:
#train = train.drop(credit_noise.index)

드랍했더니 성능이 소폭 하락했다. 즉슨 무언가 데이터에 중요한 의미가 있다는 뜻.

In [ ]:
credit_noise['credit'].value_counts(normalize=True).mul(100)

2.0    61.685568
1.0    24.193907
0.0    14.120525
Name: credit, dtype: float64

즉슨 너어무 드랍해버리고 싶지만... 신용등급이 너무 높으신 분들이 많다는 것이 문제였다. 즉 이상치라고만 처리하기엔 경향성 분석에 중요한 희귀값인 친구들이라는 뜻으로도 해석될 것 같다...

In [ ]:
a = 0
b = []
c = []
for x in credit_noise['DAYS_BIRTH'].unique():
  for y in credit_noise['income_total'].unique():
    a = credit_noise[credit_noise['DAYS_BIRTH']==x]
    b.append(a[a['income_total']==y])
    #출생일과 수입이 같으면 중복데이터라고 처리하기(너무 많은 값을 검사하면 시간이 오래 걸리고
    #그렇다고 query를 쓰자고 하니 코드 중복이 많아져서 디버깅하는 시간이 부족한 나에게는 모자람.)
for x in range(len(b)):
  if len(b[x])!= 0:
    c.append(x)
d = [] 
for x in c:
  d.append(b[x])

In [ ]:
d[2].credit.value_counts().index

Float64Index([1.0, 2.0], dtype='float64')

In [ ]:
import copy

In [ ]:
m = copy.deepcopy(d)

In [ ]:
d = copy.deepcopy(m)

In [ ]:
for x in range(len(d)):
  for y in reversed(d[x].credit.value_counts().index): #각 행의 credit 별 갯수를 카운트한 인덱스에서
    bnk=0
    mom = 0
    if d[x].credit.value_counts()[y] > bnk: # 각 값의 갯수가 적어도 0보다는 크므로 갯수에 대해서
      bnk = d[x].credit.value_counts()[y] # 전 인덱스의 값보다 더 크게 된다면
      mom = y # 그 행의 값을 받아와서
  d[x]['credit']= mom # 그 행에 대해 일괄적으로 가장 많은 값을 입력하라

최대한 데이터를 긍정적으로 평가해보았더니, 드랍한 것 보다 성능이 안 나왔다. ( 갯수 많은 중복이 있으면 그걸 선택하되, 동일하다면 좋은 쪽 선택 )

최대한 부정적으로 데이터를 평가해보자.(갯수 많은 중복이 있으면 그걸 선택하되, 동일하다면 낮은 쪽 선택 ) -> 성능 소폭 개선됨

In [ ]:
credit_distilled = pd.concat(d)

In [ ]:
for x in credit_distilled.index:
  train.loc[x] = credit_distilled.loc[x]

###이제 begin_month를 기준으로 발생한 노이즈값들을 살펴볼 차례는 스킵~




In [ ]:
begin_noise = train.loc[check_duplicate(train, 'begin_month')]

('begin_month',)을(를) 제외하고 중복되는 데이터 개수는 20564 입니다
중복 데이터 비율: 77.73



In [ ]:
begin_noise_t = test.loc[check_duplicate(test, 'begin_month')]

('begin_month',)을(를) 제외하고 중복되는 데이터 개수는 6877 입니다
중복 데이터 비율: 68.77



In [ ]:
d = group_same(begin_noise)

In [ ]:
e = group_same(begin_noise_t)

In [ ]:
len(d[0])

5

In [ ]:
train['card_num'] = 0

In [ ]:
test['card_num'] = 0

여전히 중복데이터이긴 하나... 발급받은 카드의 갯수가 될 것이라고 가정하고 카드 갯수 feature을 추가해주었다. predictor가 중복데이터간의 연관성을 파악할 수 있게 도와줄 수 있을 것으로 도움을 주기 위해서가 주 목적으로, 중복데이터를 아예 없앴을 때 발생한 문제점을 해결하기 위한 것. -> 문제가 발생했는데, 성능이 대폭 하락한다. 예측 도중 성능은 상승하는데 이유를 잘 모르겠다. 테스트셋과 트레인셋이 다르게 배치 될 여지가 있어서 그런가? 그럴 수 있는건가?

In [ ]:
for x in range(len(d)):
  awef =len(d[x])
  d[x]['card_num'] = awef

In [ ]:
awef = 0
for x in range(len(e)):
  awef =len(e[x])
  e[x]['card_num'] = awef

In [ ]:
begin_distill = pd.concat(d)

In [ ]:
begin_distill_t = pd.concat(e)

In [ ]:
begin_distill.index

Int64Index([    0,  1773, 11402, 17641, 20389,     1,  2199,  5167,  9605,
            12550,
            ...
            24925, 26295, 24997, 25404, 25131, 25681, 25569, 25736, 26328,
            26345],
           dtype='int64', name='index', length=20564)

In [ ]:
train.loc[begin_distill.index] = begin_distill

In [ ]:
test.loc[begin_distill_t.index] = begin_distill_t

###Collecting Extreamly abnormal data

In [ ]:
train[(train['family_size']-train['child_num'])<1]['credit'].value_counts(normalize=True).mul(100)

2.0    100.0
Name: credit, dtype: float64

In [ ]:
train[train['DAYS_EMPLOYED']==0][train[train['DAYS_EMPLOYED']==0]['DAYS_BIRTH']<16491]['credit'].value_counts(normalize=True).mul(100)

2.0    61.25
0.0    23.75
1.0    15.00
Name: credit, dtype: float64

In [ ]:
train['house_type'].unique()

array(['Municipal apartment', 'House / apartment', 'With parents',
       'Co-op apartment', 'Rented apartment', 'Office apartment'],
      dtype=object)

In [ ]:
dun = train[train['reality']=='N'][train[train['reality']=='N']['house_type']=='With parents']

In [ ]:
dun[dun['family_size']<2]['credit'].value_counts(normalize=True).mul(100) # 가족이랑 같이 산다고 그랬는데 가족 구성원이 본인 한 명

2.0    61.988304
1.0    32.163743
0.0     5.847953
Name: credit, dtype: float64

In [ ]:
dun[dun['family_size']>3]['credit'].value_counts(normalize=True).mul(100) #조부모 등의 가족이 포함되어있지 않은데, 가족 구성원이 본인 포함 4명 이상

2.0    68.041237
1.0    21.649485
0.0    10.309278
Name: credit, dtype: float64

In [ ]:
dun[dun['family_size']>4] #부모와 같이 사는데 가족구성원 5 이상 -> 자식이 있는데 부모와 같이 산다? 그런데 자식 수가 3명인 것을 보아 본인과 부모 한 명과 자식 3명인 것 같다.

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,work_phone,phone,email,occyp_type,family_size,begin_month,credit
index,,,,,,,,,,,,,,,,,,
987,F,N,N,3,103500.0,Working,Secondary / secondary special,Married,With parents,13674,4491,0,0,0,Laborers,5.0,420.0,2.0
4628,F,N,N,3,103500.0,Working,Secondary / secondary special,Married,With parents,13674,4491,0,0,0,Laborers,5.0,210.0,2.0
5730,M,N,N,3,126000.0,Working,Secondary / secondary special,Civil marriage,With parents,12072,1422,0,0,1,others,5.0,1350.0,2.0
7105,M,N,N,3,135000.0,Working,Secondary / secondary special,Married,With parents,11757,658,0,0,0,Laborers,5.0,810.0,2.0
11130,M,N,N,3,135000.0,Working,Secondary / secondary special,Married,With parents,11757,658,0,0,0,Laborers,5.0,840.0,2.0
11157,M,N,N,3,126000.0,Working,Secondary / secondary special,Civil marriage,With parents,12072,1422,0,0,1,others,5.0,720.0,2.0
19223,M,N,N,3,126000.0,Working,Secondary / secondary special,Civil marriage,With parents,12072,1422,0,0,1,others,5.0,60.0,1.0
19669,M,N,N,3,126000.0,Working,Secondary / secondary special,Civil marriage,With parents,12072,1422,0,0,1,others,5.0,-0.0,1.0
20395,M,N,N,3,58500.0,Working,Secondary / secondary special,Married,With parents,16087,1000,1,1,0,Laborers,5.0,1470.0,2.0


In [ ]:
wd=train[train['family_type']=='Married']
wd[wd['family_size']-wd['child_num']<2]

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,work_phone,phone,email,occyp_type,family_size,begin_month,credit
index,,,,,,,,,,,,,,,,,,
14900,M,Y,N,2,225000.0,Working,Secondary / secondary special,Married,House / apartment,14776,2212,0,0,0,Drivers,1.0,900.0,2.0
16791,F,N,Y,1,126000.0,Working,Secondary / secondary special,Married,House / apartment,12140,779,0,0,0,Core staff,1.0,1110.0,2.0
21096,M,N,N,1,157500.0,Working,Secondary / secondary special,Married,House / apartment,16424,1458,0,0,0,Drivers,1.0,750.0,2.0


In [ ]:
def abnorm_collec(data):
    ab = []
    df = copy.deepcopy(data)
    
    ab.append(df[(df['family_size']-df['child_num'])<1])    #자식수와 가족수가 같거나 자식수가 더 많음
    
    ab.append(df[df['child_num']>5])                        #자식의 수가 5명보다 많음
    
    wd = df[df['family_type']=='Single / not married']      #싱글인데
    ab.append(wd[wd['family_size']-wd['child_num']>1])      #동거인이 2명 이상임
    
    wd = df[df['family_type']=='Married']                   #결혼했는데
    ab.append(wd[wd['family_size']-wd['child_num']<2])      #동거인이 없음

    wd = df[df['family_type']=='Civil marriage']            #동거한다는데
    ab.append(wd[wd['family_size']-wd['child_num']<2])      #동거인이 없음

    ab.append(df[df['family_size']<1]) #가족 크기가 1보다 작음

    ab.append(df[(df['family_size']-df['child_num'])>3]) #동거를 3명 이상 함


    fs = df[df['family_type']=='Single / not married'][df[df['family_type']=='Single / not married']['child_num']==0]
    #싱글이고
    ab.append(fs[fs['family_size']>1])
    #동거인이 있음

    a = df[df['family_type']=='Single / not married'][df[df['family_type']=='Single / not married']['family_size']>2]
    #싱글이고 가족 크기가 3 이상인데
    ab.append(a[a['reality']== 'N'][a[a['reality']== 'N']['DAYS_EMPLOYED']<2555])
    #직장 근속을 7년 이상을 못 하고 집이 없음

    ab.append(train[train['income_type']=='Pensioner'][train[train['income_type']=='Pensioner']['occyp_type']!='beksu'])
    #일하면서 연금받음

    wd = df[df['occyp_type']=='Drivers']
    a = wd[wd['car']=='N'][wd[wd['car']=='N']['income_type']!='Commercial associate']
    ab.append(a[a['DAYS_EMPLOYED']<200])
    #드라이버인데, 차가 없고 어디 소속되어있지도 않으며 최소고용보장 이상 일하지 못함(6개월)


    abnorm = pd.concat(ab)
    abnorm.drop_duplicates(inplace=True)
    return abnorm
    


In [ ]:
train = train.drop(abnorm_collec(train).index)

In [ ]:
train = train.drop(train[train['begin_month']==0][train[train['begin_month']==0]['credit']==2].index)

### review the result

In [ ]:
train

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,work_phone,phone,email,occyp_type,family_size,begin_month,credit
index,,,,,,,,,,,,,,,,,,
0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,13899,4709,0,0,0,others,2.0,180.0,1.0
1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,11380,1540,0,0,1,Laborers,3.0,150.0,0.0
2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,19087,4434,0,1,0,Managers,2.0,660.0,1.0
3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,15088,2092,0,1,0,Sales staff,2.0,1110.0,0.0
4,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,15037,2105,0,0,0,Managers,2.0,780.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,F,N,N,2,225000.0,State servant,Secondary / secondary special,Married,House / apartment,12079,1984,0,0,0,Core staff,4.0,60.0,1.0
26453,F,N,Y,1,180000.0,Working,Higher education,Separated,House / apartment,15291,2475,0,0,0,others,2.0,1410.0,2.0
26454,F,Y,N,0,292500.0,Working,Secondary / secondary special,Civil marriage,With parents,10082,2015,0,0,0,Core staff,2.0,750.0,2.0


In [ ]:
train.credit.value_counts(normalize=True).mul(100)

2.0    63.505329
1.0    24.051195
0.0    12.443475
Name: credit, dtype: float64

In [ ]:
check_duplicate(train, 'credit')

('credit',)을(를) 제외하고 중복되는 데이터 개수는 1425 입니다
중복 데이터 비율: 5.75



Int64Index([    1,     2,    21,    24,    25,    34,    50,    53,    77,
               78,
            ...
            26346, 26354, 26369, 26386, 26391, 26397, 26403, 26430, 26431,
            26432],
           dtype='int64', name='index', length=1425)

오전 11시 38분 중복제거를 하지 않고 최대한 test의 분포에 맞게 해보았다.

중복 제거 완료

In [ ]:
len (train_c)-len(train)

38

온전히 중복 데이터를 삭제한 train_c에서 716개 정도의 데이터가 결손되었다.

##Data Preprocessing and distillation with SDV(TVAE, CTGAN, SMOTE) - TVAE SELECTED
######Using data pregenerated because of long fitting interval

데이터를 최대한 정제(cleansing)한 후에 저장한 일종의 synthetic data table이므로 sampling하여 normalizing 하는데 사용

## (Unactivated)Data Recleansing for synthetic data

##Data Spliting and Loading DATASETS for K-FOLD -- 1st way

In [ ]:
trains = fold(train,10)

In [ ]:
a = 0
for x in range(len(trains)):
  a += len(trains[x])
a

24768

## Data Spliting by Occyp_type and Apply Synthetic for remove relation caused by data bias. -- 2nd way

#Trainning Model

## K-Fold

In [ ]:
(predictors, ldbd) = fold_models(trains)

Beginning AutoGluon training ...
AutoGluon will save models to /contect/Autogluon/
AutoGluon Version:  0.0.16b20201214
Train Data Rows:    22292
Train Data Columns: 17
Preprocessing data ...
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == float, but few unique label-values observed and label-values can be converted to int).
	3 unique label values:  [0.0, 1.0, 2.0]
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Train Data Class Count: 3
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    8956.61 MB
	Train Data (Original)  Memory Usage: 13.42 MB (0.1% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators

Predictive performance on given dataset: log_loss = 0.6840990625994403


Persisting 6 models in memory. Models will require 6.53% of memory.
Fitting model: NeuralNetMXNet_FULL ...
	68.35s	 = Training runtime
Fitting model: KNeighborsUnif_FULL ...
	0.05s	 = Training runtime
Fitting model: KNeighborsDist_FULL ...
	0.05s	 = Training runtime
Fitting model: RandomForestGini_FULL ...
	8.67s	 = Training runtime
Fitting model: RandomForestEntr_FULL ...
	11.87s	 = Training runtime
Fitting model: ExtraTreesGini_FULL ...
	6.53s	 = Training runtime
Fitting model: ExtraTreesEntr_FULL ...
	7.38s	 = Training runtime
Fitting model: LightGBM_FULL ...
	4.35s	 = Training runtime
Fitting model: LightGBMXT_FULL ...
	6.15s	 = Training runtime
Fitting model: CatBoost_FULL ...
	128.03s	 = Training runtime
Fitting model: XGBoost_FULL ...
	28.21s	 = Training runtime
Fitting model: LightGBMCustom_FULL ...
	7.88s	 = Training runtime
Fitting model: WeightedEnsemble_FULL_L1 ...
	-0.6911	 = Validation log_loss score
	0.67s	 = Training runtime
	0.0s	 = Validation runtime
Beginning AutoGlu

Predictive performance on given dataset: log_loss = 0.7219664866009935


Models will not be persisted in memory as they are expected to require 13.81% of memory, which is greater than the specified max_memory limit of 10.0%.
	Models will be loaded on-demand from disk to maintain safe memory usage, increasing inference latency. If inference latency is a concern, try to use smaller models or increase the value of max_memory.
Fitting model: NeuralNetMXNet_FULL ...
	68.98s	 = Training runtime
Fitting model: KNeighborsUnif_FULL ...
	0.05s	 = Training runtime
Fitting model: KNeighborsDist_FULL ...
	0.06s	 = Training runtime
Fitting model: RandomForestGini_FULL ...
	8.66s	 = Training runtime
Fitting model: RandomForestEntr_FULL ...
	11.97s	 = Training runtime
Fitting model: ExtraTreesGini_FULL ...
	6.17s	 = Training runtime
Fitting model: ExtraTreesEntr_FULL ...
	7.47s	 = Training runtime
Fitting model: LightGBM_FULL ...
	4.47s	 = Training runtime
Fitting model: LightGBMXT_FULL ...
	6.18s	 = Training runtime
Fitting model: CatBoost_FULL ...
	118.11s	 = Training ru

...

Models will not be persisted in memory as they are expected to require 21.79% of memory, which is greater than the specified max_memory limit of 10.0%.
	Models will be loaded on-demand from disk to maintain safe memory usage, increasing inference latency. If inference latency is a concern, try to use smaller models or increase the value of max_memory.
Fitting model: NeuralNetMXNet_FULL ...
	53.65s	 = Training runtime
Fitting model: KNeighborsUnif_FULL ...
	0.05s	 = Training runtime
Fitting model: KNeighborsDist_FULL ...
	0.05s	 = Training runtime
Fitting model: RandomForestGini_FULL ...
	8.57s	 = Training runtime
Fitting model: RandomForestEntr_FULL ...
	11.87s	 = Training runtime
Fitting model: ExtraTreesGini_FULL ...
	6.29s	 = Training runtime
Fitting model: ExtraTreesEntr_FULL ...
	7.59s	 = Training runtime
Fitting model: LightGBM_FULL ...
	2.8s	 = Training runtime
Fitting model: LightGBMXT_FULL ...
	5.07s	 = Training runtime
Fitting model: CatBoost_FULL ...
	77.38s	 = Training runt

Predictive performance on given dataset: log_loss = 0.6952480542802993


Models will not be persisted in memory as they are expected to require 27.49% of memory, which is greater than the specified max_memory limit of 10.0%.
	Models will be loaded on-demand from disk to maintain safe memory usage, increasing inference latency. If inference latency is a concern, try to use smaller models or increase the value of max_memory.
Fitting model: NeuralNetMXNet_FULL ...
	70.36s	 = Training runtime
Fitting model: KNeighborsUnif_FULL ...
	0.06s	 = Training runtime
Fitting model: KNeighborsDist_FULL ...
	0.05s	 = Training runtime
Fitting model: RandomForestGini_FULL ...
	8.67s	 = Training runtime
Fitting model: RandomForestEntr_FULL ...
	11.97s	 = Training runtime
Fitting model: ExtraTreesGini_FULL ...
	6.18s	 = Training runtime
Fitting model: ExtraTreesEntr_FULL ...
	7.59s	 = Training runtime
Fitting model: LightGBM_FULL ...
	3.19s	 = Training runtime
Fitting model: LightGBMXT_FULL ...
	4.38s	 = Training runtime
Fitting model: CatBoost_FULL ...
	105.2s	 = Training run

Predictive performance on given dataset: log_loss = 0.703029465811378


Models will not be persisted in memory as they are expected to require 16.55% of memory, which is greater than the specified max_memory limit of 10.0%.
	Models will be loaded on-demand from disk to maintain safe memory usage, increasing inference latency. If inference latency is a concern, try to use smaller models or increase the value of max_memory.
Fitting model: NeuralNetMXNet_FULL ...
	68.88s	 = Training runtime
Fitting model: KNeighborsUnif_FULL ...
	0.05s	 = Training runtime
Fitting model: KNeighborsDist_FULL ...
	0.05s	 = Training runtime
Fitting model: RandomForestGini_FULL ...
	8.68s	 = Training runtime
Fitting model: RandomForestEntr_FULL ...
	11.88s	 = Training runtime
Fitting model: ExtraTreesGini_FULL ...
	6.19s	 = Training runtime
Fitting model: ExtraTreesEntr_FULL ...
	7.49s	 = Training runtime
Fitting model: LightGBM_FULL ...
	2.57s	 = Training runtime
Fitting model: LightGBMXT_FULL ...
	4.99s	 = Training runtime
Fitting model: CatBoost_FULL ...
	125.84s	 = Training ru

## Occyp_type

## abnormal data

#Inference Model

## Inferencing each models

In [ ]:
predictors

In [ ]:
ldbd

[                       model  score_test  ...  can_infer  fit_order
 0   WeightedEnsemble_FULL_L1   -0.675648  ...       True         26
 1        WeightedEnsemble_L1   -0.684099  ...       True         13
 2        LightGBMCustom_FULL   -0.689813  ...       True         25
 3             LightGBMCustom   -0.692859  ...       True         12
 4              LightGBM_FULL   -0.706017  ...       True         21
 5                   LightGBM   -0.708045  ...       True          8
 6               XGBoost_FULL   -0.708836  ...       True         24
 7      RandomForestGini_FULL   -0.711343  ...       True         17
 8           RandomForestEntr   -0.715397  ...       True          5
 9                    XGBoost   -0.718564  ...       True         11
 10           LightGBMXT_FULL   -0.720969  ...       True         22
 11     RandomForestEntr_FULL   -0.721902  ...       True         18
 12                LightGBMXT   -0.724585  ...       True          9
 13          RandomForestGini   -0

## ensemble

/content/AutogluonModels


ag-20210520_034147/  ag-20210520_042546/  ag-20210520_050345/
ag-20210520_035217/  ag-20210520_043605/  ag-20210520_051121/
ag-20210520_040246/  ag-20210520_044500/
ag-20210520_041351/  ag-20210520_045555/


In [ ]:
awef = []
for x in range(len(predictors)):
  for y in range(len(ldbd[x])):
    if ldbd[x]['model'][y] == "WeightedEnsemble_FULL_L1" or ldbd[x]['model'][y] == "WeightedEnsemble_L1" or ldbd[x]['model'][y] == "LightGBMCustom_FULL":
      awef.append(y)
      break

In [ ]:
awef

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
kf = []
ite = 0
for x in range(len(predictors)):
  print(ldbd[x]['model'][awef[x]])
  if ldbd[x]['model'][awef[x]] == "RandomForestGini_FULL":
    pass
  else:
    kf.append(predictors[x].predict_proba(test, model = ldbd[x]['model'][awef[x]], as_pandas=True, as_multiclass=True))
    ite += 1

WeightedEnsemble_FULL_L1
WeightedEnsemble_FULL_L1
WeightedEnsemble_FULL_L1
WeightedEnsemble_FULL_L1
WeightedEnsemble_FULL_L1
WeightedEnsemble_FULL_L1
WeightedEnsemble_FULL_L1
WeightedEnsemble_FULL_L1
WeightedEnsemble_FULL_L1
WeightedEnsemble_FULL_L1


In [ ]:
kf = []
for x in range(len(predictors)):
  kf.append(predictors[x].predict_proba(test, model = ldbd[x]['model'][0], as_pandas=True, as_multiclass=True))

Int64Index([31375, 34703, 26701, 27141, 27273, 30423, 30486, 30562, 31647,
            32065, 32462, 34224, 35699, 26680, 29157, 30180, 30211, 31211,
            36021],
           dtype='int64', name='index')

In [ ]:
a = 0
for x in range(ite):
  a += kf[x]

In [ ]:
a

,0.0,1.0,2.0
index,,,
26457,0.423114,1.101500,8.475386
26458,3.966882,1.763964,4.269154
26459,0.467913,0.749952,8.782135
26460,1.198020,0.965532,7.836447
26461,0.749377,1.944211,7.306413
...,...,...,...
36452,0.944189,4.595446,4.460365
36453,1.917490,4.667751,3.414759
36454,0.176703,0.660081,9.163217


In [ ]:
k = a.sort_index()
k.head(10)

,0.0,1.0,2.0
index,,,
26457,0.423114,1.101500,8.475386
26458,3.966882,1.763964,4.269154
26459,0.467913,0.749952,8.782135
26460,1.198020,0.965532,7.836447
26461,0.749377,1.944211,7.306413
26462,0.384085,0.740432,8.875484
26463,3.907071,5.938968,0.153960
26464,0.651161,0.869707,8.479134
26465,0.560777,1.032261,8.406962


# FINISHED!!!!

In [ ]:
cd /content/drive/MyDrive/mtfk/results

/content/drive/MyDrive/mtfk/results


In [ ]:
a = sub.copy()

In [ ]:
sub['0'] = k[0]
sub['1'] = k[1]
sub['2'] = k[2]

In [ ]:
k = (sub + a)/ite

In [ ]:
test_t = abnorm_collec(test)

In [ ]:
abn = k.loc[test_t.index]

In [ ]:
for x in range(len(k.loc[test_t.index])):
  print(x)
  abn.iloc[x]['0'] = 0.00
  abn.iloc[x]['1'] = 0.00
  abn.iloc[x]['2'] = 1.00
abn.iloc[x]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


0    0.0
1    0.0
2    1.0
Name: 35063, dtype: float64

In [ ]:
k.loc[abn.index] = abn

In [ ]:
k.loc[test_t.index]

,0,1,2
index,,,
31375,0.0,0.0,1.0
34703,0.0,0.0,1.0
26701,0.0,0.0,1.0
27141,0.0,0.0,1.0
27273,0.0,0.0,1.0
30423,0.0,0.0,1.0
30486,0.0,0.0,1.0
30562,0.0,0.0,1.0
31647,0.0,0.0,1.0


In [ ]:
k.head(10)

,0,1,2
index,,,
26457,0.042311,0.110150,0.847539
26458,0.396688,0.176396,0.426915
26459,0.046791,0.074995,0.878214
26460,0.119802,0.096553,0.783645
26461,0.074938,0.194421,0.730641
26462,0.038409,0.074043,0.887548
26463,0.390707,0.593897,0.015396
26464,0.065116,0.086971,0.847913
26465,0.056078,0.103226,0.840696


In [ ]:
k.to_csv('K_fold_auto_10times_reshape_log_loss_abnremoved.csv')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
a

,0,1,2
index,,,
26457,0,0,0
26458,0,0,0
26459,0,0,0
26460,0,0,0
26461,0,0,0
...,...,...,...
36452,0,0,0
36453,0,0,0
36454,0,0,0


In [ ]:
a = pd.read_csv(output_directory+"/K_fold_auto_13times.csv",index_col=index)
b = pd.read_csv(output_directory+"/Old_beksu.csv",index_col=index)
d = pd.read_csv(output_directory+"/Longest_iter.csv",index_col=index)

In [ ]:
e = (a + b  + d )/3

In [ ]:
for x in range(len(e.loc[test_t.index])):
  print(x)
  abn.iloc[x]['0'] = 0.00
  abn.iloc[x]['1'] = 0.00
  abn.iloc[x]['2'] = 1.00
abn.iloc[x]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


0    0.0
1    0.0
2    1.0
Name: 35063, dtype: float64

In [ ]:
e.loc[abn.index] = abn

In [ ]:
e.to_csv("This_is_my_last.csv")